In [1]:
%pylab inline
%load_ext autoreload
%autoreload 2

Populating the interactive namespace from numpy and matplotlib


In [2]:
import numpy as np


In [10]:
from hyperopt import Trials, STATUS_OK, tpe
from hyperas import optim
from hyperas.distributions import choice, uniform, conditional, loguniform
from keras.models import Sequential
from keras.layers import Dense, Activation, Convolution1D, Flatten, MaxPooling1D, Lambda, Convolution2D, Flatten, Reshape, LSTM, Dropout, TimeDistributed, Permute
from keras.regularizers import l2
from keras.optimizers import Adam

In [5]:
datapath = '/media/sf_VBox_Shared/timeseries/UCR_WaveGesture/'

In [13]:
def data(datapath, n=100):
    X_train = np.load(datapath+'X_train.npy')
    y_train_binary = np.load(datapath+'y_train_binary.npy')
    X_val = np.load(datapath+'X_val.npy')
    y_val_binary = np.load(datapath+'y_val_binary.npy')
    X_test = np.load(datapath+'X_test.npy')
    y_test_binary = np.load(datapath+'y_test_binary.npy')
    return X_train[:n,:,:], y_train_binary[:n,:,:], X_test[:n,:,:], y_val_binary[:n,:]

In [12]:
def model(X_train, y_train, X_test, y_val):
    _, dim_length, dim_channels = X_train.shape # number of samples in a time series
    _, outputdim = y_train.shape # number of classes
   
    min_layers=1 
    max_layers=10,
    filter_sizes = [16, 32, 64, 128]
    min_fc_nodes=10
    max_fc_nodes=100
    lr_low = -4
    lr_high = -1
    reg_low =  -4
    reg_high = -1
    nr_epochs = 5

    num_layers = {{choice(list(range(min_layers, max_layers+1)))}}
    learning_rate = {{loguniform(lr_low, lr_high)}}
    regularization_rate = {{loguniform(reg_low, reg_high)}}
    
    model = Sequential()
    filter_0 = {{choice(filter_sizes)}}
    model.add(
        Convolution1D(filters[0], 3, border_mode='same', input_shape=(dim_length, dim_channels),
                      W_regularizer=l2(regularization_rate)))
    for l in range(1, num_layers):
        filter_l = {{choice(filter_sizes)}}
        model.add(Convolution1D(filter_number, 3, border_mode='same', W_regularizer=l2(regularization_rate)))
        model.add(Activation('relu'))
    model.add(Flatten())
    fc_hidden_nodes = {{choice(list(range(min_fc_nodes, max_fc_nodes+1)))}}
    model.add(Dense(output_dim=fc_hidden_nodes, W_regularizer=l2(regularization_rate)))  # Fully connected layer
    model.add(Activation('relu'))  # Relu activation
    model.add(Dense(output_dim=outputdim))
    model.add(Activation("softmax"))  # Final classification layer

    model.compile(loss='categorical_crossentropy',
                  optimizer=Adam(lr=learning_rate),
                  metrics=['accuracy'])
    history = model.fit(X_train, y_train,
                            nb_epoch=nr_epochs, batch_size=20, # see comment on subsize_set
                            validation_data=(X_val, y_val),
                            show_accuracy=True,
                            verbose=2)
    score, acc = model.evaluate(X_val, y_val, verbose=0)
    print('Test accuracy:', acc)
    return {'loss': -acc, 'status': STATUS_OK, 'model': model}

In [14]:
best_run, best_model = optim.minimize(model=model,
                                          data=lambda: data(datasets),
                                          algo=tpe.suggest,
                                          max_evals=5,
                                          trials=Trials())

PARTS:
num_layers
learning_rate
regularization_rate
filter_0
filter_0
fc_hidden_nodes
>>> Hyperas search space:

def get_space():
    return {
        'num_layers': hp.choice('num_layers', list(range(min_layers, max_layers+1))),
        'learning_rate': hp.loguniform('learning_rate', lr_low, lr_high),
        'regularization_rate': hp.loguniform('regularization_rate', reg_low, reg_high),
        'filter_0': hp.choice('filter_0', filter_sizes),
        'filter_0_1': hp.choice('filter_0_1', filter_sizes),
        'fc_hidden_nodes': hp.choice('fc_hidden_nodes', list(range(min_fc_nodes, max_fc_nodes+1))),
    }

>>> Data



>>> Resulting replaced keras model:

def keras_fmin_fnct(space):

    _, dim_length, dim_channels = X_train.shape # number of samples in a time series
    _, outputdim = y_train.shape # number of classes
   
    min_layers=1 
    max_layers=10,
    filter_sizes = [16, 32, 64, 128]
    min_fc_nodes=10
    max_fc_nodes=100
    lr_low = -4
    lr_high = -1
    reg_low =  -

SyntaxError: invalid syntax (temp_model.py, line 1)